In [2]:
%pip install sentencepiece pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\tanma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import sentencepiece as spm
import pandas as pd
train_df = pd.read_csv(r"data\eng.sentence.train.tsv", sep="\t", header=None)
training_txt = train_df[0].values.tolist()  + train_df[1].values.tolist()
training_txt = [str(sent) for sent in training_txt]
with open('data\\training_ulm.txt', 'w', encoding='utf-8') as file:
  file.writelines(f"{line}\n" for line in training_txt)
spm.SentencePieceTrainer.train('--input=data\\training_ulm.txt --model_prefix=models\m_ulm --vocab_size=1800 --model_type=unigram')
sp_ulm = spm.SentencePieceProcessor()
sp_ulm.load('models\m_ulm.model')

test_df = pd.read_csv(r"data\eng.sentence.test.tsv", sep="\t", header=None)
encoded = []
test_list = test_df[0].values.tolist()
i = 0
for sentence in test_list:
  enc = sp_ulm.encode_as_pieces(sentence)
  encoded.append(enc)
  print(enc)
  i += 1
  if i > 100:
    break

['▁What', '▁', 'if', '▁Go', 'o', 'g', 'le', '▁M', 'or', 'ph', 'ed', '▁In', 'to', '▁Go', 'o', 'g', 'le', 'O', 'S', '▁?']
['▁What', '▁', 'if', '▁Go', 'o', 'g', 'le', '▁', 'exp', 'and', 'ed', '▁on', '▁it', 's', '▁', 'search', '▁-', '▁engine', '▁(', '▁and', '▁now', '▁e', '-', 'mail', '▁)', '▁', 'ware', 's', '▁into', '▁a', '▁full', '▁-', '▁fl', 'ed', 'g', 'ed', '▁opera', 't', 'ing', '▁system', '▁?']
['▁', '[', '▁vi', 'a', '▁M', 'ic', 'ro', 's', 'o', 'f', 't', '▁W', 'at', 'ch', '▁', 'from', '▁Mar', 'y', '▁J', 'o', '▁F', 'ol', 'e', 'y', '▁', ']']
['▁(', '▁And', '▁,', '▁by', '▁the', '▁way', '▁,', '▁is', '▁any', 'body', '▁else', '▁just', '▁a', '▁litt', 'le', '▁no', 'st', 'al', 'g', 'ic', '▁for', '▁the', '▁day', 's', '▁when', '▁tha', 't', '▁was', '▁a', '▁good', '▁thing', '▁?', '▁)']
['▁Thi', 's', '▁B', 'u', 'z', 'z', 'M', 'a', 'ch', 'ine', '▁post', '▁argue', 's', '▁tha', 't', '▁Go', 'o', 'g', 'le', "▁'", 's', '▁', 'ru', 'sh', '▁to', 'ward', '▁', 'ub', 'i', 'q', 'u', 'ity', '▁m', 'ight', '▁back',

In [4]:
sp_ulm.encode_as_pieces("He's killed a man gruesomely")

['▁He', "'", 's', '▁kill', 'ed', '▁a', '▁man', '▁gr', 'u', 'e', 'some', 'ly']

In [7]:
df_guess = pd.read_csv('data\eng.sentence.test.gold.tsv', sep='\t', header=None)
i = 0
for entry in test_list:
  sent = ""
  list_sent = sp_ulm.encode_as_pieces(entry)
  list_sent = [str(x) for x in list_sent]
  # list_sent = [w.replace("▁", "") for w in list_sent]
  list_sent = [item for item in list_sent if item != ""]
  j = 0
  length = len(list_sent)  
  for morph in list_sent:
    if morph == "▁":
      list_sent[j + 1] = "▁" + list_sent[j + 1]
      j +=1
      continue 
    elif "▁" in morph and j==0: 
      sent += morph.replace("▁", "")
    elif "▁" in morph and j!=0:
      sent+= morph.replace("▁", " ")
    else:
      sent += (" @@" + morph)  
    j +=1  
  df_guess[1][i] = sent
  i+=1

df_guess.to_csv("outputs\eng.sentence.test.ulm_guess.tsv", sep='\t', header=None, index = False)